# AdventureWorks: resit questions

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)

········


In [2]:
cust_aw = pd.read_sql_table('CustomerAW', engine)
cust_addr = pd.read_sql_table('CustomerAddress', engine)
addr = pd.read_sql_table('Address', engine)
product = pd.read_sql_table('Product', engine)
order_det = pd.read_sql_table('SalesOrderDetail', engine)
order_head = pd.read_sql_table('SalesOrderHeader', engine)
prod_model = pd.read_sql_table('ProductModel', engine)
prod_model_prod = pd.read_sql_table('ProductModelProductDescription', engine)
prod_desc = pd.read_sql_table('ProductDescription', engine)
prod_cat = pd.read_sql_table('ProductCategory', engine)

## 1.
**List the SalesOrderNumber for the customer 'Good Toys' 'Bike World'**

In [3]:
(order_head.merge(cust_aw.loc[cust_aw['CompanyName'].isin(['Good Toys', 'Bike World'])], 
                  on='CustomerID', how='right'))[['SalessOrderNumber', 'CompanyName']]

,SalessOrderNumber,CompanyName
0,NaN,Bike World
1,SO71774,Good Toys


## 2.
**List the ProductName and the quantity of what was ordered by 'Futuristic Bikes'**

In [4]:
(product.merge(order_det, on='ProductID', how='right')
 .merge(order_head, on='SalesOrderID', how='right')
 .merge(cust_aw.loc[cust_aw['CompanyName']=='Futuristic Bikes'], 
        on='CustomerID', how='right'))[['Name', 'OrderQty']]

,Name,OrderQty
0,ML Mountain Seat/Saddle,2.0
1,"Long-Sleeve Logo Jersey, L",2.0
2,"Classic Vest, S",3.0


## 3.
**List the name and addresses of companies containing the word 'Bike' (upper or lower case) and companies containing 'cycle' (upper or lower case). Ensure that the 'bike's are listed before the 'cycles's.**

In [5]:
a = (cust_aw.merge(cust_addr, on='CustomerID')
     .merge(addr, on='AddressID')
     [['CompanyName', 'AddressLine1', 'AddressLine2', 'City',
       'StateProvince', 'CountryRegion', 'PostalCode']])
a['flag'] = None
a.loc[a['CompanyName'].str.lower().str.contains('bike'), 'flag'] = 0
a.loc[a['CompanyName'].str.lower().str.contains('cycle'), 'flag'] = 1
a.dropna(subset='flag').sort_values(['flag', 'CompanyName'])

,CompanyName,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,flag
0,A Bike Store,2251 Elliot Avenue,None,Seattle,Washington,United States,98104,0
177,A Typical Bike Shop,"One Dancing, Rr",No. 25 Box 8033,Round Rock,Texas,United States,78664,0
3,Advanced Bike Components,12345 Sterling Avenue,None,Irving,Texas,United States,75061,0
322,Area Bike Accessories,6900 Sisk Road,None,Modesto,California,United States,95354,0
8,Associated Bikes,5420 West 22500 South,None,Salt Lake City,Utah,United States,84101,0
...,...,...,...,...,...,...,...,...
217,"Unicycles, Bicycles, and Tricycles",Stonewood Mall,None,Downey,California,United States,90241,1
324,Valley Bicycle Distributors,5867 Sunrise Boulevard,None,Citrus Heights,California,United States,95610,1
353,Valley Bicycle Specialists,Blue Ridge Mall,None,Kansas City,Missouri,United States,64106,1
72,Westside Cycle Store,25550 Executive Dr,None,Elgin,Illinois,United States,60120,1


## 4.
**Show the total order value for each CountryRegion. List by value with the highest first.**

In [6]:
a = addr.merge(order_head, left_on='AddressID', right_on='ShipToAddressID')
(a.groupby('CountryRegion')['SubTotal'].sum()
 .reset_index().sort_values('SubTotal', ascending=False))

,CountryRegion,SubTotal
0,United Kingdom,518096.42
1,United States,347336.69


## 5.
**Find the best customer in each region.**

In [7]:
t = (addr.merge(order_head, left_on='AddressID', right_on='ShipToAddressID')
     .merge(cust_aw, on='CustomerID'))
a = (t.groupby(['CountryRegion', 'CompanyName'])['SubTotal']
     .sum().reset_index())
(a.sort_values('SubTotal', ascending=False).groupby('CountryRegion')
 .apply(lambda i: i.iloc[0]))

,CountryRegion,CompanyName,SubTotal
CountryRegion,,,
United Kingdom,United Kingdom,Action Bicycle Specialists,108561.83
United States,United States,Eastside Department Store,83858.43
